In [3]:
val order_schema = "order_id int, order_date date, customer_id int, order_status string"

order_schema = order_id int, order_date date, customer_id int, order_status string


order_id int, order_date date, customer_id int, order_status string

In [5]:
val order_df = spark.read.format("csv").schema(order_schema).load("/public/trendytech/orders/orders_1gb.csv")

Waiting for a Spark session to start...

order_df = [order_id: int, order_date: date ... 2 more fields]


[order_id: int, order_date: date ... 2 more fields]

In [6]:
order_df.createOrReplaceTempView("order")

In [8]:
spark.sql("""
select order_id*1 as new_order_id,order_status from (select order_id,customer_id
,order_status from order where order_id<500) where order_id<200
""").explain(true)

== Parsed Logical Plan ==
'Project [('order_id * 1) AS new_order_id#13, 'order_status]
+- 'Filter ('order_id < 200)
   +- 'SubqueryAlias `__auto_generated_subquery_name`
      +- 'Project ['order_id, 'customer_id, 'order_status]
         +- 'Filter ('order_id < 500)
            +- 'UnresolvedRelation `order`

== Analyzed Logical Plan ==
new_order_id: int, order_status: string
Project [(order_id#0 * 1) AS new_order_id#13, order_status#3]
+- Filter (order_id#0 < 200)
   +- SubqueryAlias `__auto_generated_subquery_name`
      +- Project [order_id#0, customer_id#2, order_status#3]
         +- Filter (order_id#0 < 500)
            +- SubqueryAlias `order`
               +- Relation[order_id#0,order_date#1,customer_id#2,order_status#3] csv

== Optimized Logical Plan ==
Project [(order_id#0 * 1) AS new_order_id#13, order_status#3]
+- Filter ((isnotnull(order_id#0) && (order_id#0 < 500)) && (order_id#0 < 200))
   +- Relation[order_id#0,order_date#1,customer_id#2,order_status#3] csv

== Physica